# Lesson-7

### Вспомогательные функции:

Модуль:

In [1]:
def abs(number):
    return number if number > 0 else -number

Реализация матриц и операций над ними:

In [2]:
class Matrix:
    
    class_round = 10 #Порядок округления

    def __init__(self, array, n_lines, m_columns):
        self.array = array
        self.n_lines = n_lines
        self.m_columns = m_columns

    @classmethod
    def copy_array(cls, array): #Копирование
        n_lines = len(array)
        m_columns = len(array[0])
        new_array = [[0 for _ in range(m_columns)] for _ in range(n_lines)]
        for i in range(n_lines):
            for j in range(m_columns):
                new_array[i][j] = array[i][j]
        return new_array

    @classmethod
    def transpose(cls, matrix): #Транцонирование
        new_array = [[matrix.array[i][j] for i in range(matrix.n_lines)] for j in range(matrix.m_columns)]
        new_matrix = cls(new_array, matrix.m_columns, matrix.n_lines)
        return new_matrix

    @classmethod
    def sum(cls, matrix1, matrix2): #Сложение
        if (matrix1.n_lines != matrix2.n_lines) or (matrix1.m_columns != matrix2.m_columns):
            print("ERROR: (matrix1.n_lines != matrix2.n_lines) or (matrix1.m_columns != matrix2.m_columns)")
            return None
        new_array = [[matrix1.array[i][j] + matrix2.array[i][j] for j in range(matrix1.m_columns)] for i in
                     range(matrix1.n_lines)]
        new_matrix = cls(new_array, matrix1.n_lines, matrix1.m_columns)
        return new_matrix
    
    @classmethod
    def dot_scalar(cls, matrix, scalar): #Умножение на скаляр
        new_array = [[(matrix.array[i][j] * scalar) for j in range(matrix.m_columns)] for i in range(matrix.n_lines)]
        new_matrix = cls(new_array, matrix.n_lines, matrix.m_columns)
        return new_matrix

    @classmethod
    def dot(cls, matrix1, matrix2): #Умножение
        if matrix1.m_columns != matrix2.n_lines:
            print("ERROR: matrix1.m_columns != matrix2.n_lines")
            return None
        new_array = [[0 for _ in range(matrix2.m_columns)] for _ in range(matrix1.n_lines)]
        for i in range(matrix1.n_lines):
            for j in range(matrix2.m_columns):
                new_array[i][j] = sum([matrix1.array[i][k] * matrix2.array[k][j] for k in range(matrix1.m_columns)])
        new_matrix = cls(new_array, matrix1.n_lines, matrix2.m_columns)
        return new_matrix

    @classmethod
    def det(cls, origin_array): #Определитель
        if len(origin_array) != len(origin_array[0]):
            print("ERROR: matrix.n_lines != matrix.m_columns")
            return None
        array: list = cls.copy_array(origin_array)
        length = len(array)
        sign = 0
        for j in range(length - 1):
            if array[j][j] == 0:
                for i in range(j + 1, length):
                    if array[i][j] != 0:
                        array[i], array[j] = array[j], array[i]
                        sign += 1
                        break
            for i in range(j + 1, length):
                if array[i][j] != 0:
                    array[i] = [x - y for x, y in zip(array[i], [k * (array[i][j] / array[j][j]) for k in array[j]])]
        det = (-1) ** sign
        for i in range(length):
            det *= array[i][i]
        return det

    @classmethod
    def inv(cls, matrix): #Обратная матрица
        if matrix.n_lines != matrix.m_columns:
            print("ERROR: matrix.n_lines != matrix.m_columns")
            return None
        a_det = cls.det(matrix.array)
        if a_det == 0:
            print("ERROR: a_det == 0")
            return None
        if matrix.n_lines == 1:
            return cls([[1 / a_det]], matrix.n_lines, matrix.m_columns)
        new_array = cls.copy_array(matrix.array)
        for i in range(matrix.n_lines):
            for j in range(matrix.m_columns):
                new_array[i][j] = ((-1) ** (i + j)) * cls.det(
                    [[matrix.array[q][w] for w in range(matrix.m_columns) if w != j] for q in range(matrix.n_lines) if
                     q != i]) / a_det
        new_matrix = cls.transpose(cls(new_array, matrix.n_lines, matrix.m_columns))
        return new_matrix

    def __str__(self):
        data = ""
        for line in self.array:
            for k in line:
                data += str(round(k, self.class_round)) + " "
            data += "\n"
        return data[:-1]

Функции из математической статистики:

In [3]:
class Math_Stat:
    
    @classmethod
    def mean(cls, array): #Среднее арифметическое
        sum = 0
        for x in array:
            sum += x
        return sum / len(array)
    
    @classmethod
    def var(cls, array, ddof = 0): #Дисперсия
        sum = 0
        mean = cls.mean(array) 
        for x in array:
            sum += (x - mean)**2
        return sum / (len(array) - ddof)
    
    @classmethod
    def std(cls, array, ddof = 0): #Среднее квадратичное отклонение
        return cls.var(array, ddof)**(0.5)
    
    @classmethod
    def covcoef(cls, array_1, array_2): #Коэфициент ковариации
        return cls.mean([array_1[i] * array_2[i] for i in range(len(array_1))]) - cls.mean(array_1) * cls.mean(array_2)
    
    @classmethod
    def corrcoef(cls, array_1, array_2): #Коэфициент корреляции Пирсона
        return cls.covcoef(array_1, array_2) / (cls.std(array_1) * cls.std(array_2))
    
    @classmethod
    def lin_reg_matrix(cls, array_x, array_y, intercept = True): #Коэфициенты линейной регресси через матрицы
        if intercept:
            matrix_x = Matrix([[1, n] for n in array_x], len(array_x), 2)
        else:
            matrix_x = Matrix.transpose(Matrix([array_x], 1, len(array_x)))
        matrix_y = Matrix.transpose(Matrix([array_y], 1, len(array_y)))
        return Matrix.dot(Matrix.dot(Matrix.inv(Matrix.dot(Matrix.transpose(matrix_x), matrix_x)), Matrix.transpose(matrix_x)), 
                          matrix_y)
    
    @classmethod
    def mse_lin_reg(cls, B0, B1, array_x, array_y): #Средняя квадратическая ошибка линейной регресси
        n = len(array_x)
        return sum([(B0 + array_x[i] * B1 - array_y[i])**2 for i in range(n)]) / n
    
    @classmethod #Коэфициенты линейной регресси через градиентный спуск
    def lin_reg_grad(cls, array_x, array_y, alpha = 10**(-5), intercept = True):
        n = len(array_x)
        B0 = 0
        B1 = 1
        last_mse = -1
        new_mse = cls.mse_lin_reg(B0, B1, array_x, array_y)
        if intercept:
            B1_old = B1
            while abs(new_mse - last_mse) > alpha:
                B1 -= alpha * (2/n) * sum([((B0 + array_x[i] * B1_old - array_y[i]) * array_x[i]) for i in range(n)])
                B0 -= alpha * (2/n) * sum([(B0 + array_x[i] * B1_old - array_y[i]) for i in range(n)])
                B1_old = B1
                last_mse = new_mse
                new_mse = cls.mse_lin_reg(B0, B1, array_x, array_y)
            return [B0, B1]
        else:
            while abs(new_mse - last_mse) > alpha:
                B1 -= alpha * (2/n) * sum([((array_x[i] * B1 - array_y[i]) * array_x[i]) for i in range(n)])
                last_mse = new_mse
                new_mse = cls.mse_lin_reg(B0, B1, array_x, array_y)
            return B1

## 1. Даны значения величины заработной платы заемщиков банка (zp) и значения их поведенческого кредитного скоринга (ks): zp = [35, 45, 190, 200, 40, 70, 54, 150, 120, 110], ks = [401, 574, 874, 919, 459, 739, 653, 902, 746, 832]. Используя математические операции, посчитать коэффициенты линейной регрессии, приняв за X заработную плату (то есть, zp - признак), а за y - значения скорингового балла (то есть, ks - целевая переменная). Произвести расчет как с использованием intercept, так и без.

In [4]:
zp = [35, 45, 190, 200, 40, 70, 54, 150, 120, 110]
ks = [401, 574, 874, 919, 459, 739, 653, 902, 746, 832]

Коэффициенты линейной регрессии используя матрицы (с intercept(b0, b1)):

In [5]:
res = Math_Stat.lin_reg_matrix(zp, ks)
print(res)

444.1773573244 
2.6205388824 


Коэффициенты линейной регрессии используя матрицы (без intercept(b1)):

In [6]:
res = Math_Stat.lin_reg_matrix(zp, ks, intercept = False)
print(res)

5.8898204201 


## 2. Посчитать коэффициент линейной регрессии при заработной плате (zp), используя градиентный спуск (без intercept).

In [7]:
zp = [35, 45, 190, 200, 40, 70, 54, 150, 120, 110]
ks = [401, 574, 874, 919, 459, 739, 653, 902, 746, 832]

Коэффициент линейной регрессии используя градиентный спуск (без intercept):

In [8]:
res = Math_Stat.lin_reg_grad(zp, ks, intercept = False)
res

5.889796976614508

## 3. В каких случаях для вычисления доверительных интервалов и проверки статистических гипотез используется таблица значений функции Лапласа, а в каких - таблица критических точек распределения Стьюдента? 

Таблица значений функции Лапласа используется для вычисления доверительных интервалов и проверки статистических гипотез, когда нам известна дисперсия или среднее квадратическое отклонение генеральной совокупности.

Таблица критических точек распределения Стьюдента используется для вычисления доверительных интервалов и проверки статистических гипотез, когда нам НЕизвестна дисперсия или среднее квадратическое отклонение генеральной совокупности.

## 4. Произвести вычисления как в пункте 2, но с вычислением intercept. Учесть, что изменение коэффициентов должно производиться на каждом шаге одновременно (то есть изменение одного коэффициента не должно влиять на изменение другого во время одной итерации).

In [9]:
zp = [35, 45, 190, 200, 40, 70, 54, 150, 120, 110]
ks = [401, 574, 874, 919, 459, 739, 653, 902, 746, 832]

Коэффициент линейной регрессии используя градиентный спуск (с intercept):

In [10]:
res = Math_Stat.lin_reg_grad(zp, ks)
res

[442.2062074383119, 2.6350474170595026]